In [16]:
#you will need this on JupyterHub every time, just once on your own system
!pip install folium

In [17]:
import pandas as pd
import folium

## Plotting points from coordinates in data

In [ ]:
#Data from gis.vb.opendata.gov

In [18]:
fire = pd.read_csv('data/fire_bomb.csv')
fire.head()

,Incident Type,Exposure Number,Battalion,Station Area,Action Taken - Primary,Location 1,Coordinates,Latitude,Longitude
0,Bomb scare - no bomb,0,3.0,ST19,Investigate,"5000 FERRELL PW\nVIRGINIA BEACH, VA 23464-\n(3...","[36.795457, -76.165328]",36.795457,-76.165328
1,Bomb scare - no bomb,0,2.0,ST07,Investigate,"4800 COLUMBUS ST\nVIRGINIA BEACH, VA 23462-\n(...","[36.841082, -76.141192]",36.841082,-76.141192
2,Bomb scare - no bomb,0,2.0,ST07,Investigate,"5100 CLEVELAND ST\nVIRGINIA BEACH, VA 23462-\n...","[36.842197, -76.160385]",36.842197,-76.160385
3,Bomb scare - no bomb,0,3.0,ST19,"Information, investigation & enforcement, other","900 AVERY WY\nVIRGINIA BEACH, VA 23464-\n(36.7...","[36.769781, -76.176491]",36.769781,-76.176491
4,Bomb scare - no bomb,0,NaN,ST11,Investigate,"800 VIRGINIA BEACH BL\nVIRGINIA BEACH, VA 2345...","[36.843333, -75.985762]",36.843333,-75.985762


In [22]:
fire["Location 1"].value_counts()

3600 SENTARA WY\nVIRGINIA BEACH, VA 23452-\n(36.834006, -76.096595)           7
2400 NIMMO PW\nVIRGINIA BEACH, VA 23456-\n(36.755972, -76.061489)             6
4500 VIRGINIA BEACH BL\nVIRGINIA BEACH, VA 23462-\n(36.843585, -76.131499)    3
5100 CLEVELAND ST\nVIRGINIA BEACH, VA 23462-\n(36.842197, -76.160385)         3
1000 INDEPENDENCE BL\nVIRGINIA BEACH, VA 23455-\n(36.872251, -76.133985)      2
                                                                             ..
4400 BONNEY RD\nVIRGINIA BEACH, VA 23462-\n(36.835922, -76.127379)            1
2400 TOURNAMENT DR\nVIRGINIA BEACH, VA 23456-\n(36.759584, -76.074787)        1
3500 CLUB HOUSE RD\nVIRGINIA BEACH, VA 23452-\n(36.824549, -76.091047)        1
4800 SHORE DR\nVIRGINIA BEACH, VA 23455-\n(36.904467, -76.135929)             1
5300 LAKE LAWSON RD\nVIRGINIA BEACH, VA 23455-\n(36.886165, -76.160951)       1
Name: Location 1, Length: 162, dtype: int64

In [23]:
fire['Incident Type'].unique()

array(['Bomb scare - no bomb',
       'Explosive, bomb removal (for bomb scare, use 721)',
       'Munitions or bomb explosion (no fire)'], dtype=object)

In [24]:
recode = {'Bomb scare - no bomb': 'Bomb Scare', 'Explosive, bomb removal (for bomb scare, use 721)':'Bomb Removal',
         'Munitions or bomb explosion (no fire)': 'Explosion'}
fire['Incident Type'] = fire['Incident Type'].apply(lambda x: recode[x])

In [25]:
fire['Incident Type'].unique()

array(['Bomb Scare', 'Bomb Removal', 'Explosion'], dtype=object)

In [26]:
fire = fire.rename(columns={'Action Taken - Primary':'Action', 'Location 1':'Location'})

In [27]:
#first, how do we make a map?
m = folium.Map(tiles = 'CartoDB positron')
m

In [29]:
#all our data is from Virginia Beach, so let's center on that
VA_BEACH = [36.8516,-75.9792]
m = folium.Map(VA_BEACH, zoom_start = 12)
m

In [30]:
#let's add a marker on the map for every item in the df
#then we will add a popup and tooltip (it's extraneous, just by way of example)
VA_BEACH = [36.8516,-75.9792]
m = folium.Map(VA_BEACH, zoom_start = 12)
for i in range(fire.shape[0]):
    lat,lon = fire.loc[i,'Latitude'],fire.loc[i,'Longitude']
    incident = fire.loc[i,'Incident Type']
    action = fire.loc[i, 'Action'] 
    folium.Marker([lat,lon], popup = incident, tooltip = action).add_to(m)
m

In [35]:
#let's use circle markers instead with colors
colors = {'Bomb Scare':'grey', 'Bomb Removal':'blue', 'Explosion':'red'}
#copy-paste our map/loop code here
VA_BEACH = [36.8516,-75.9792]
m = folium.Map(VA_BEACH, zoom_start = 12)
for i in range(fire.shape[0]):
    lat,lon = fire.loc[i,'Latitude'],fire.loc[i,'Longitude']
    incident = fire.loc[i,'Incident Type']
    action = fire.loc[i, 'Action'] 
    color = colors[incident]
    folium.CircleMarker([lat,lon], 
                  tooltip = action, fill_color = color, color = color, 
                    radius = 3, fill_opacity = 1).add_to(m)

m
        

In [ ]:
#let's make the FeatureGroups
#m = #copy from above
VA_BEACH = [36.8516,-75.9792]
m = folium.Map(VA_BEACH, zoom_start = 12)

colors = {'Bomb Scare':'grey', 'Bomb Removal':'blue', 'Explosion':'red'}
key_list = list(colors.keys())
groups = [folium.FeatureGroup(name = f'<span style="color: {colors[k]};\
font-size:large">{k}</span>') for k in key_list]
    
for g in range(len(groups)):
    m.add_child(groups[g]) #add each group as a child of the map
    cur_obs = fire[fire['Incident Type'] == key_list[g]].reset_index(drop = True)
    for i in range(cur_obs.shape[0]):
        lat,lon = cur_obs.loc[i,'Latitude'],cur_obs.loc[i,'Longitude']
        incident = cur_obs.loc[i,'Incident Type']
        action = cur_obs.loc[i, 'Action'] 
        color = colors[incident]
        folium.CircleMarker([lat,lon], 
                  tooltip = action, fill_color = color, color = color, 
                    radius = 3, fill_opacity = 1).add_to(groups[g])

title_text = 'Bomb-related incidents in Virginia Beach'
title_html = f'<h3 align="center" style="font-size:24px;font-weight:bold">{title_text}</h3>'
m.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl(collapsed = False).add_to(m)
m.save('fire_bomb.html')
m

In [ ]:
#copy previous here

#Add title
title_text = 'Bomb-related incidents in Virginia Beach'
title_html = f'<h3 align="center" style="font-size:24px;font-weight:bold">{title_text}</h3>'
m.get_root().html.add_child(folium.Element(title_html))

#Save it
m.save('bomb.html')
m